# Agrupamento de features categóricas - Experimento

Este componente realiza o agrupamento de features categóricas com alta cardinalidade para serem implementadas por modelos de classificação. Neste componente são implementadas três estratégias de agrupamento (method):

* percent: o agrupamento é feito considerando um ponto de corte definido. Categorias abaixo desse limiar são agrupadas em uma categoria denominada "other";

* top_n: apenas as n categorias mais frequentes são mantidas. As demais são agrupadas em uma categoria denominada "other";

* kmeans: Para cada classe, são calculadas sua frequência relativa e porcentagem de casos positivos da variável resposta. Então, o algoritmo kmeans utiliza estas informações para o agrupamento. 


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
dataset = "/tmp/data/eucalyptus.csv" #@param {type:"string"}
target = "Utility" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

high_cardinality_features = "Sp" #@param {type:"feature",multiple:true,label:"Features categóricas para fazer agrupamento de classes",description:"O agrupamento será feito considerando apenas as features categóricas selecionadas neste parâmetro."}

method = "kmeans" #@param ["percent","top_n","kmeans"] {type:"string",multiple:false,label:"Estratégia de agrupamento",description:"Escolha do método que será utilizado para agrupar as classes das variávevis categóricas. "}

threshold = 0.1 #@param {type:"number", label:"ponto de corte para agrupamento (utilizado apenas com estratégia 'percent')", description:"Classes com frequência relativa abaixo desse ponto de corte serão agrupadas em uma mesma classe"}
n = 10 #@param {type:"number", label:"Número de classes que deverão ser mantidas/criadas (utilizado apenas com estratégia 'top_n' e 'kmeans')", description:"Se a estratégia for kmeans, n representa o número de grupos criados pelo algoritmo, se for top_n, as n classes mais frequentes serão mantidas"}

## Acesso ao conjunto de dados

Utiliza a função `load_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.<br>
O tipo da variável retornada depende do arquivo de origem:

- [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e *compressed* CSV: `.csv` `.csv.zip` `.csv.gz` `.csv.bz2` `.csv.xz`
- [`Binary IO stream`](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: `.jpg` `.wav` `.zip` `.h5` `.parquet` etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)
X = df.drop(target, axis=1)
y = df[target]

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados.<br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)

# check if the target column is numerical or categorical
if featuretypes[target_index] == "Categorical":
    task = "classification"
else:
    task = "regression"
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [ ]:
df.dropna(subset=[target], inplace=True)
y = df[target].to_numpy()

## Codifica labels do atributo alvo

As labels do atributo alvo são convertidos em números inteiros ordinais com valor entre 0 e n_classes-1.

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarize = LabelBinarizer()
y = label_binarize.fit_transform(y)

## Configuração das features

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Find the position of categorical features with high cardinality
high_cardinality_features = np.asarray(high_cardinality_features)
high_cardinality_indexes = np.where(
    ~(featuretypes == NUMERICAL) & np.isin(columns, high_cardinality_features)
)[0]

remaining_indexes = np.setdiff1d(range(len(featuretypes)), high_cardinality_indexes)
# After the step grouping_high_cardinality,
# the selected categorical features are grouped in the beggining of the array
high_cardinality_indexes_after_pipeline = np.arange(len(high_cardinality_indexes))
remaining_indexes_after_pipeline = np.arange(
    len(high_cardinality_indexes), len(featuretypes)
)

## Implementação do agrupamento

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/projects/master/samples/grouping-categorical-features/group_feat.py

In [ ]:
from group_feat import GroupCatFeatures

X = X.to_numpy()
g = GroupCatFeatures(
    method=method,
    n=n,
    threshold=threshold,
    column_index=high_cardinality_indexes,
    column_name=columns[high_cardinality_indexes],
    task=task,
)

# Train model and transform dataset
X_res, dict_class = g.fit_transform(X, y)

features_after_pipeline = np.concatenate(
    (columns[high_cardinality_indexes], columns[remaining_indexes])
)

# Put data back in a pandas.DataFrame
df_res = pd.DataFrame(data=X_res, columns=features_after_pipeline)

y = label_binarize.inverse_transform(y)
df_res[target] = y

## Salva alterações no conjunto de dados

Utiliza a função `save_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar alterações no conjuntos de dados.

In [ ]:
# save dataset changes
df_res.to_csv(dataset, index=False)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "columns": columns,
    "columns_changed": columns_changed,
    "features_after_pipeline": features_after_pipeline,
    "dict_class": dict_class,
}

dump(artifacts, "/tmp/data/grouping-categorical-features.joblib")